# Adv Games Regression
<a id="top"></a>

Treating games as the sum of each team's players stats.
0. [Helper Methods](#helpers)
1. [Score Differential Regressor](#score-diff)
    * [SVD regression](#score-diff-lr), [RFT regression](#score-diff-logr)
    * Only take raw game scores and betting lines
2. [Past Win% Differential Regressor](#winper)
    * [SVD regression](#winper-lr), [RFT regression](#winper-logr)
    * Take past win% over 10, 25, 50, 75, 100 games for each team
3. [Box Score Regressor](#boxscore)
    * Takes in overall box score for either team 

In [50]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [141]:
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.options.display.float_format = '{:5}'.format

## Helper Methods
[to top](#top)
<a id="helpers">

In [52]:
def SVGRegression(X_input, y_input):
    n_folds = 5
    kf = KFold(n_folds, shuffle=True)

    # Create a pipeline that first standardizes the data, then applies PCA, and finally trains the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components="mle")),  # k is the number of principal components
        ('regressor', SVR())
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_input, np.ravel(y_input), cv=kf, scoring='neg_mean_squared_error')

    # Calculate average MSE
    average_mse = -np.mean(cv_scores)
    print("cv:", cv_scores)
    print("Average MSE:", average_mse)
    print("Average RMSE:", np.sqrt(average_mse))

    return pipeline

def SVGRegressionFit(X_input, y_input):

    X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=.2)

    # Create a pipeline that first standardizes the data, then applies PCA, and finally trains the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components="mle")),  # k is the number of principal components
        ('regressor', SVR())
    ])

    pipeline.fit(X_train, np.ravel(y_train))
    # y_pred = pipeline.predict(X_test)

    # # The coefficients
    # # The mean squared error
    # print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    # # The coefficient of determination: 1 is perfect prediction
    # print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

    return pipeline

## Rolling Regressors
[to top](#top)

-  [linear regression](#winper-lr)
-  [logistic regression](#winper-logr)
<a id="winper">

In [136]:
df = pd.read_csv("data_proc/games_rolling_master.csv")
df.dropna(inplace=True)
# display(df)
game_info = ['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_line', 'home_prob', 'home_line', 'away_spread', 'away_total'] 
intervals = ['10g_', '25g_', '50g_', '75g_', '100g']
columns = game_info + [c for c in df.columns.to_list() if c not in game_info and c[:4] in intervals]
print(columns)

['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_line', 'home_prob', 'home_line', 'away_spread', 'away_total', '10g_FGM_T1', '10g_FGM_T2', '10g_FGA_T1', '10g_FGA_T2', '10g_FG3M_T1', '10g_FG3M_T2', '10g_FG3A_T1', '10g_FG3A_T2', '10g_FTM_T1', '10g_FTM_T2', '10g_FTA_T1', '10g_FTA_T2', '10g_OREB_T1', '10g_OREB_T2', '10g_DREB_T1', '10g_DREB_T2', '10g_REB_T1', '10g_REB_T2', '10g_AST_T1', '10g_AST_T2', '10g_STL_T1', '10g_STL_T2', '10g_BLK_T1', '10g_BLK_T2', '10g_TO_T1', '10g_TO_T2', '10g_PF_T1', '10g_PF_T2', '10g_PTS_T1', '10g_PTS_T2', '10g_PLUS_MINUS_T1', '10g_PLUS_MINUS_T2', '10g_FG_PCT_T1', '10g_FG_PCT_T2', '10g_FG3_PCT_T1', '10g_FG3_PCT_T2', '10g_FT_PCT_T1', '10g_FT_PCT_T2', '10g_SCORE_DIFF_T1', '10g_SCORE_DIFF_T2', '10g_WON_T1', '10g_WON_T2', '25g_FGM_T1', '25g_FGM_T2', '25g_FGA_T1', '25g_FGA_T2', '25g_FG3M_T1', '25g_FG3M_T2', '25g_FG3A_T1', '25g_FG3A_T2', '25g_FTM_T1', '25g_FTM_T2', '25g_FTA_T1', '25g_FTA_T2', '25g_OREB_T1', '25g_OREB_T2', '25g_DREB_T1', '25g_DREB_T2', '25g_REB_T

In [137]:
away_spread_reg = SVGRegressionFit(df.iloc[:-1000][columns].values, df[:-1000]["SCORE_DIFF_T1"].values)
                
# SVGRegression(df[['TEAM1_ID', 'TEAM2_ID', 'DATE', 'TEAM1_10wp', 'TEAM1_25wp', 'TEAM1_50wp', 'TEAM1_75wp', 'TEAM1_100wp', 'TEAM2_10wp', 'TEAM2_25wp', 'TEAM2_50wp', 'TEAM2_75wp', 'TEAM2_100wp']],df[["SCORE_DIFF_T1"]])

# SVGRegression(df.drop(columns=['SCORE_DIFF_T1', 'SCORE_DIFF_T2', '']), df[["SCORE_DIFF_T1"]])
# fig = px.scatter(df, x="away_spread", y="SCORE_DIFF", hover_data=["SBR_GAME_ID"], trendline="ols")
# fig.show()


In [147]:
buffer = 6

def calcOutcomes(i):
    away_spread_reg = SVGRegressionFit(df.iloc[:-i][columns].values, df[:-i]["SCORE_DIFF_T1"].values)
    
    result = pd.DataFrame()    
    result['SBR_GAME_ID'] = df.iloc[-i:]['SBR_GAME_ID']
    result['pred_away_diff'] = away_spread_reg.predict(df.iloc[-i:][columns].values)
    result['bet_away_spread'] = df.iloc[-i:]['away_spread']
    result['diff_spread'] = result['pred_away_diff'] + result['bet_away_spread']
    result['actual_away_diff'] = df.iloc[-i:]['SCORE_DIFF_T1']
    result['away_line'] = df.iloc[-i:]['away_line']
    result['home_line'] = df.iloc[-i:]['home_line']
    result['WON_T1'] = df.iloc[-i:]['WON_T1']
    result['WON_T2'] = df.iloc[-i:]['WON_T2']
    
    result['bet_decision'] = np.abs(result['diff_spread']) > buffer
    result.loc[result['diff_spread'] > 0, 'bet_on'] = 1
    result.loc[result['diff_spread'] < 0, 'bet_on'] = 2

    result.loc[(result['bet_on'] == 1) & (result['away_line'] > 0), 'profit'] = result['away_line'] + 100
    result.loc[(result['bet_on'] == 1) & (result['away_line'] < 0), 'profit'] = 100 * (100 / - result['away_line']) + 100
    result.loc[(result['bet_on'] == 1) & (result['actual_away_diff'] + result['bet_away_spread'] < 0), 'profit'] = -100 # 

    result.loc[(result['bet_on'] == 2) & (result['home_line'] > 0), 'profit'] = result['home_line'] - 100
    result.loc[(result['bet_on'] == 2) & (result['home_line'] < 0), 'profit'] = 100 * (100 / - result['home_line']) + 100
    result.loc[(result['bet_on'] == 2) & (result['actual_away_diff'] + result['bet_away_spread'] > 0), 'profit'] = -100

    result['bet_on'] = result['bet_on'] * result['bet_decision']
    result['profit'] = result['profit'] * result['bet_decision']

    display(result)
    return np.average(result['profit'])

# for i in range(100):
#     testIndex(i)
calcOutcomes(2000)

,SBR_GAME_ID,pred_away_diff,bet_away_spread,diff_spread,actual_away_diff,away_line,home_line,WON_T1,WON_T2,bet_decision,bet_on,profit
7648,20220124_nba_Chicago_Oklahoma City,3.1880596274681,-1.0,2.1880596274681,1.0,-115.0,-105.0,1.0,0.0,False,0.0,0.0
7649,20220124_nba_Indiana_New Orleans,-3.271800074887834,4.0,0.728199925112166,-4.0,150.0,-170.0,0.0,1.0,False,0.0,0.0
7650,20220124_nba_New York_Cleveland,-4.1499614143969135,5.0,0.8500385856030865,-2.0,175.0,-210.0,0.0,1.0,False,0.0,0.0
7651,20220124_nba_Utah_Phoenix,-7.9921455769923195,12.0,4.0078544230076805,-6.0,500.0,-700.0,0.0,1.0,False,0.0,0.0
7652,20220125_nba_Dallas_Golden State,-4.992670701670468,3.5,-1.492670701670468,-38.0,135.0,-155.0,0.0,1.0,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9643,20231110_nba_Philadelphia_Detroit,6.828105121102803,-8.5,-1.6718948788971968,8.0,-387.5,302.5,1.0,0.0,False,0.0,0.0
9644,20231110_nba_Brooklyn_Boston,-6.373770351360639,12.0,5.626229648639361,-14.0,472.5,-674.895031490553,0.0,1.0,False,0.0,-0.0
9645,20231110_nba_L.A. Clippers_Dallas,0.59150184039179,1.0,1.59150184039179,-18.0,-103.498950021,-116.497700091996,0.0,1.0,False,0.0,-0.0
9646,20231110_nba_L.A. Lakers_Phoenix,-2.023342602885735,2.0,-0.023342602885735086,3.0,107.5,-127.5,1.0,0.0,False,0.0,-0.0


17.4385000001

In [71]:
df = pd.read_csv("data_proc/games_rolling_master.csv")
df

,GAME_ID,SBR_GAME_ID,DATE,TEAM_ID_T1,FGM_T1,FGA_T1,FG3M_T1,FG3A_T1,FTM_T1,FTA_T1,OREB_T1,DREB_T1,REB_T1,AST_T1,STL_T1,BLK_T1,TO_T1,PF_T1,PTS_T1,PLUS_MINUS_T1,FG_PCT_T1,FG3_PCT_T1,FT_PCT_T1,SCORE_DIFF_T1,WON_T1,TEAM_ID_T2,FGM_T2,FGA_T2,FG3M_T2,FG3A_T2,FTM_T2,FTA_T2,OREB_T2,DREB_T2,REB_T2,AST_T2,STL_T2,BLK_T2,TO_T2,PF_T2,PTS_T2,PLUS_MINUS_T2,FG_PCT_T2,FG3_PCT_T2,FT_PCT_T2,SCORE_DIFF_T2,WON_T2,away_line,home_line,away_prob,home_prob,away_spread,away_total,10g_FGM_T1,10g_FGM_T2,10g_FGA_T1,10g_FGA_T2,10g_FG3M_T1,10g_FG3M_T2,10g_FG3A_T1,10g_FG3A_T2,10g_FTM_T1,10g_FTM_T2,10g_FTA_T1,10g_FTA_T2,10g_OREB_T1,10g_OREB_T2,10g_DREB_T1,10g_DREB_T2,10g_REB_T1,10g_REB_T2,10g_AST_T1,10g_AST_T2,10g_STL_T1,10g_STL_T2,10g_BLK_T1,10g_BLK_T2,10g_TO_T1,10g_TO_T2,10g_PF_T1,10g_PF_T2,10g_PTS_T1,10g_PTS_T2,10g_PLUS_MINUS_T1,10g_PLUS_MINUS_T2,10g_FG_PCT_T1,10g_FG_PCT_T2,10g_FG3_PCT_T1,10g_FG3_PCT_T2,10g_FT_PCT_T1,10g_FT_PCT_T2,10g_SCORE_DIFF_T1,10g_SCORE_DIFF_T2,10g_WON_T1,10g_WON_T2,25g_FGM_T1,25g_FGM_T2,25g_FGA_T1,25g_FGA_T2,25g_FG3M_T1,25g_FG3M_T2,25g_FG3A_T1,25g_FG3A_T2,25g_FTM_T1,25g_FTM_T2,25g_FTA_T1,25g_FTA_T2,25g_OREB_T1,25g_OREB_T2,25g_DREB_T1,25g_DREB_T2,25g_REB_T1,25g_REB_T2,25g_AST_T1,25g_AST_T2,25g_STL_T1,25g_STL_T2,25g_BLK_T1,25g_BLK_T2,25g_TO_T1,25g_TO_T2,25g_PF_T1,25g_PF_T2,25g_PTS_T1,25g_PTS_T2,...,50g_FGM_T2,50g_FGA_T1,50g_FGA_T2,50g_FG3M_T1,50g_FG3M_T2,50g_FG3A_T1,50g_FG3A_T2,50g_FTM_T1,50g_FTM_T2,50g_FTA_T1,50g_FTA_T2,50g_OREB_T1,50g_OREB_T2,50g_DREB_T1,50g_DREB_T2,50g_REB_T1,50g_REB_T2,50g_AST_T1,50g_AST_T2,50g_STL_T1,50g_STL_T2,50g_BLK_T1,50g_BLK_T2,50g_TO_T1,50g_TO_T2,50g_PF_T1,50g_PF_T2,50g_PTS_T1,50g_PTS_T2,50g_PLUS_MINUS_T1,50g_PLUS_MINUS_T2,50g_FG_PCT_T1,50g_FG_PCT_T2,50g_FG3_PCT_T1,50g_FG3_PCT_T2,50g_FT_PCT_T1,50g_FT_PCT_T2,50g_SCORE_DIFF_T1,50g_SCORE_DIFF_T2,50g_WON_T1,50g_WON_T2,75g_FGM_T1,75g_FGM_T2,75g_FGA_T1,75g_FGA_T2,75g_FG3M_T1,75g_FG3M_T2,75g_FG3A_T1,75g_FG3A_T2,75g_FTM_T1,75g_FTM_T2,75g_FTA_T1,75g_FTA_T2,75g_OREB_T1,75g_OREB_T2,75g_DREB_T1,75g_DREB_T2,75g_REB_T1,75g_REB_T2,75g_AST_T1,75g_AST_T2,75g_STL_T1,75g_STL_T2,75g_BLK_T1,75g_BLK_T2,75g_TO_T1,75g_TO_T2,75g_PF_T1,75g_PF_T2,75g_PTS_T1,75g_PTS_T2,75g_PLUS_MINUS_T1,75g_PLUS_MINUS_T2,75g_FG_PCT_T1,75g_FG_PCT_T2,75g_FG3_PCT_T1,75g_FG3_PCT_T2,75g_FT_PCT_T1,75g_FT_PCT_T2,75g_SCORE_DIFF_T1,75g_SCORE_DIFF_T2,75g_WON_T1,75g_WON_T2,100g_FGM_T1,100g_FGM_T2,100g_FGA_T1,100g_FGA_T2,100g_FG3M_T1,100g_FG3M_T2,100g_FG3A_T1,100g_FG3A_T2,100g_FTM_T1,100g_FTM_T2,100g_FTA_T1,100g_FTA_T2,100g_OREB_T1,100g_OREB_T2,100g_DREB_T1,100g_DREB_T2,100g_REB_T1,100g_REB_T2,100g_AST_T1,100g_AST_T2,100g_STL_T1,100g_STL_T2,100g_BLK_T1,100g_BLK_T2,100g_TO_T1,100g_TO_T2,100g_PF_T1,100g_PF_T2,100g_PTS_T1,100g_PTS_T2,100g_PLUS_MINUS_T1,100g_PLUS_MINUS_T2,100g_FG_PCT_T1,100g_FG_PCT_T2,100g_FG3_PCT_T1,100g_FG3_PCT_T2,100g_FT_PCT_T1,100g_FT_PCT_T2,100g_SCORE_DIFF_T1,100g_SCORE_DIFF_T2,100g_WON_T1,100g_WON_T2
0,21400003.0,20141028_nba_Houston_L.A. Lakers,20141028.0,1610612745.0,31.0,73.0,12.0,29.0,34.0,50.0,14.0,33.0,47.0,22.0,7.0,3.0,13.0,30.0,108.0,90.0,0.4246575342465753,0.4137931034482758,0.68,18.0,1.0,1610612747.0,28.0,79.0,3.0,10.0,31.0,39.0,11.0,25.0,36.0,16.0,7.0,3.0,11.0,32.0,90.0,-90.0,0.3544303797468354,0.3,0.7948717948717948,-18.0,0.0,-300.0,250.0,0.75,0.2857142857142857,-7.0,207.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na